<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (You will be implementing the decoder, not the encoder, as we are not doing sequence-to-sequence translation.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [1]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2022-10-17 00:45:57--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 54.210.151.85, 52.200.164.228, 18.207.91.3, ...
Connecting to piazza.com (piazza.com)|54.210.151.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2022-10-17 00:45:58--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.226.228.46, 13.226.228.50, 13.226.228.49, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.226.228.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  2.04MB/s 

In [4]:
chunk_len = 200
 
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
  
print(random_chunk())

will.
hopefully it will.  how come?
how come?   i like how clear the sky gets after it rains.
i like how clear the sky gets after it rains.   i feel the same way. it smells so good after it rains.
i fe


In [5]:
import torch

In [6]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please do not look at the documentation's code for the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**

**DONE:**
* Create a custom GRU cell



In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(GRU, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.lin3 = nn.Linear(input_size, hidden_size)
        self.lin4 = nn.Linear(hidden_size, hidden_size)
        self.lin5 = nn.Linear(input_size, hidden_size)
        self.lin6 = nn.Linear(hidden_size, hidden_size)

    def forward(self, inputs, hiddens):
        # Each layer does the following:
        # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
        # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
        # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
        # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
        # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)

        h_t_1 = hiddens[-1]
        r_t = F.sigmoid(self.lin1(inputs) + self.lin2(h_t_1))
        z_t = F.sigmoid(self.lin3(inputs) + self.lin4(h_t_1))
        n_t = F.tanh(self.lin5(inputs) + (r_t * self.lin6(h_t_1)))
        h_t = ((1 - z_t) * n_t) + (z_t * h_t_1)

        return h_t, hiddens.append(h_t)

        # r_t = F.sigmoid(self.lin1(inputs) + self.lin2(hiddens))
        # z_t = F.sigmoid(self.lin3(inputs) + self.lin4(hiddens))
        # n_t = F.tanh(self.lin5(inputs) + (r_t * self.lin6(hiddens)))
        # h_t = ((1 - z_t) * n_t) + (z_t * hiddens)

        # return h_t

---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**

**DONE:**
* Create an RNN class that extends from nn.Module.



In [8]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        # more stuff here...
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input_char, hidden):
        # by reviewing the documentation, construct a forward function that properly uses the output
        # of the GRU

        # stuff here
        output = F.relu(self.embedding(input_char).view(1,1,-1))
        # output.unsqueeze_(0).unsqueeze_(0)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

In [9]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**

**DONE:**
* Fill in the pieces.




In [14]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target):
    ## initialize hidden layers, set up gradient and loss 
    # your code here
    objective = nn.CrossEntropyLoss()
    decoder_optimizer.zero_grad()
    hidden = decoder.init_hidden()
    hidden = hidden.cuda()
    loss = 0

    # more stuff here...

    # calculate loss
    for i, c in enumerate(inp):
        c = c.cuda()
        y_hat, hidden = decoder(c, hidden)
        y_hat.squeeze_()
        y_truth = target[i].cuda()
        loss += objective(y_hat, y_truth)

    # calculate gradients
    with torch.autograd.set_detect_anomaly(True):
        loss.backward()

    # step the optimizer
    decoder_optimizer.step()

    # return average loss
    return loss / len(target)

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**

**DONE:**
* Fill out the evaluate function to generate text from a primed string



In [15]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # As temperature approaches 0, this sampling function becomes argmax (no randomness)
    # As temperature approaches infinity, this sampling function becomes a purely random choice
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    ## initialize hidden state, initialize other useful variables
    # your code here

    with torch.inference_mode():
        # initialize hidden
        hidden = decoder.init_hidden()
        hidden = hidden.cuda()

        n = len(prime_str)
        for i in range(predict_len):
            # prime string
            if i < n:
                c = char_tensor(prime_str[i])
                c = c.cuda()
                hidden = decoder(c, hidden)[1]

            # prediction
            else:
                c = char_tensor(prime_str[-1])
                c = c.cuda()
                output, hidden = decoder(c, hidden)
                new_c = sample_outputs(output.squeeze_(), temperature)
                prime_str += all_characters[new_c]

    return prime_str

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:** 

**DONE:**
* Create some cool output




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:
 
 Gandalf was decrond. 
'All have lord you. Forward the road at least walk this is stuff, and 
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long 
row hrough. In  

 lo:
 
 lost death it. 
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate. 
'Yes, as you there were remembaused to seen their pass, when? What 
said here, such seven an the sear 

 lo:
 
 low, and frod to keepn 
Came of their most. But here priced doubtless to an Sam up is 
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of 
the like 

 I:
 
 I had been the 
in his eyes with the perushed to lest, if then only the ring and the legended 
of the less of the long they which as the 
enders of Orcovered and smood, and the p 

 I:
 
 I they were not the lord of the hoomes. 
Home already well from the Elves. And he sat strength, and we 
housed out of the good of the days to the mountains from his perith. 

'Yess! Where though as if  

 Th:
 
 There yarden 
you would guard the hoor might. Far and then may was 
croties, too began to see the drumbred many line 
and was then hoard walk and they heart, and the chair of the 
Ents of way, might was 

 G:
 
 Gandalf 
been lat of less the round of the stump; both and seemed to the trees and perished they 
lay are speered the less; and the wind the steep and have to she 
precious. There was in the oonly went 

 wh:
 
 which went out of the door. 
Hull the King and of the The days of his brodo 
stumbler of the windard was a thing there, then it been shining langing 
to him poor land. They hands; though they seemed ou 

 ra:
 
 rather,' have all the least deather 
down of the truven beginning to the house of sunk. 
'Nark shorts of the Eyes of the Gate your great nothing as Eret. 
'I wander trust horn, and there were not, it  

 I:
 
 I can have no mind 
together! Where don't may had one may little blung 
terrible to tales. And turn and Gandalf shall be not to as only the Cattring 
not stopped great the out them forms. On they she lo 

---


In [12]:
import time

n_epochs = 5000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder = decoder.cuda()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [ ]:
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set())       
    loss_avg += loss_

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[158.75932216644287 (200 4%) 2.4543]
Wh siat 

here whir. hald ariir, mir bir'le 

Air or hiwl phe hir 
he Ros asferet Ghas thi 

ore fat 

[314.7109889984131 (400 8%) 2.2047]
When om with us ice turtares moon felcet shet herpoads theer and nomtend wond in theet and? 

I The  

[470.7207565307617 (600 12%) 1.8075]
Wh. 

Bout do the wing seet we met of cabe at we had nout sady, 
seid had of I meamen you. Nuch dear 

[628.1389856338501 (800 16%) 2.0303]
Wh we 
soger water am pipe 
the himsing and sure strime af the's the four laked and gay hinse boost  

[783.7472183704376 (1000 20%) 2.0128]
Wh it was a tiyligets.' 
But the ride, and crome again to me regore and the and 
like fire onk was a 

[939.484744310379 (1200 24%) 1.7963]
Wh will all lave 









with home Ride than Light out with on the -main,' so not was been now fea 

[1095.2329308986664 (1400 28%) 1.6973]
Whing mare they flar so. At came seemed. Frido the Wells they heart. 

But were had can have that a  

[1251.031359195709

In [ ]:
for i in range(10):
    start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
    start = random.randint(0,len(start_strings)-1)
    print(start_strings[start])
    #   all_characters.index(string[c])
    print(evaluate(start_strings[start], 200), '\n')

 ra
 raned through the 
ruses with the ports. But not suddenly rain the East of your heart with 
the trair more last. There was they has enough not back 
after a great dread night has very day flade a bea 

 he
 hed lies, tom right while their eyes and planded the 
Serves whistented and stood the high and the rustarded his slow, and has heard up. 

Suddenly to desirite by the shadows of the turred the bares  

 Th
 Thisted us every clear firest of her to the clistly 
banning fire of the brooct. Pippin listened the others, far 
which the King in the great, her eyes and strunger they seemed the else and by the ma 

 ra
 raded by the road. He was shall speak of the stopped 
pouring. There was not to be rung and pale was for so stand they same the 
courter. Frodo start in the welk place for the stone deep his cried hi 

 G
 Grandless, two the dark some 
clessed their led of the soon on the great hole-strode deep, as he said the 
Mark of the Brandir with the dell at the horse. Tomer to be r

---

## Part 6: Generate output on a different dataset

---

**TODO:**

**DONE:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)S



In [3]:
file = unidecode.unidecode(open('./normal_speech.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 249679


In [16]:
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set())       
    loss_avg += loss_

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[204.26637864112854 (200 4%) 2.3602]
Whanl nnoul alid lhol, iny.e at way yot. that.n hanr  re he'd  ne tise rous he ousinked a fom  inll  

[279.5081624984741 (400 8%) 2.2789]
Wh.
what thaces sa
tout'.    that ham.    i do you shat proulling?
int's forongens  geost?     it he 

[354.8873829841614 (600 12%) 1.5517]
Wh sass lot sfole pealnt.
ouble wiml to we worter you wlinktise picf as tusleets?
beatitte it it bei 

[430.24724316596985 (800 16%) 1.8346]
Wh a lots bet a book you some sule to chy only saiker i nem thout.
i can cousse see was do to setter 

[505.6461420059204 (1000 20%) 1.7270]
Wher proog to say didn't hat ahe wemtor, it's here could?  did you does fun tome mut think you will  

[581.0196619033813 (1200 24%) 1.4851]
Whed of him necclapfed it.
it's not? i think i wait with the fines ny.
i wold be the bill blees me u 

[656.5073094367981 (1400 28%) 1.3670]
Wher do the droft to have the like.   i got it are new that new of the card.
that time are your you  

[731.728561162948

In [17]:
for i in range(10):
    start_strings = [" What is your name?",
                     " My name is",
                     " I feel",
                     " Why do"]
    start = random.randint(0,len(start_strings)-1)
    print(start_strings[start])
    #   all_characters.index(string[c])
    print(evaluate(start_strings[start], 200), '\n')

 My name is
 My name is.
they should people warmel money in a saladly.   i need a all of money.  a can see it bases and his early.
let's been a nice a bater. that's not a really don't all his gragice.
every rabbe 

 My name is
 My name isand?
what are you like? old at the houses different pocket.
but the weather was a bark about a lot.    what kinding you had her baby one?
does man we got to be?    a bear for a money.   i t 

 Why do
 Why dor.
i was take that at some into the shoking.    what else i shouldn't rain like?
what do you mean?   i don't free your hours, but you've use for my friend.
but you're right a bank something.    

 What is your name?
 What is your name?  have years about $8, and have to go.
because they are carts at the sholling to make a would fing.   well, you're right right?
what hard about the way?    okay, more that an outer? 

 I feel
 I feely eat the collell parts.
but can be weather is so strupe.    the streets weal blow his storatimans.
the tall go out elect

I wanted to train my model on more real, modern speech. The output is okay. A lot of phrases are good, but sentences don't make sense. All the words look good though. I think with a little more training the results would be pretty good.